In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import time
from selenium.webdriver.common.by import By

url='https://www.premierleague.com/'

# get the webpage
webpage = requests.get(url)

# load webpage into bs4
soup = BeautifulSoup(webpage.content, 'html.parser')

In [6]:
import selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(ChromeDriverManager().install())
driver.maximize_window()
driver.get(url)



====== WebDriver manager ======
Current google-chrome version is 97.0.4692
Get LATEST chromedriver version for 97.0.4692 google-chrome
Driver [C:\Users\kah_chen_chan\.wdm\drivers\chromedriver\win32\97.0.4692.71\chromedriver.exe] found in cache
C:\Users\KAH_CH~1\AppData\Local\Temp/ipykernel_24212/2054763912.py:5: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install())


In [7]:
time.sleep(6) #wait for page to load

driver.find_element(By.XPATH,"/html/body/div[2]/div/div/div[1]/div[5]/button[1]").click() # accept cookies
time.sleep(2)

if len(driver.find_elements(By.ID, "advertClose"))>0:
    driver.find_element(By.XPATH,"//*[@id='advertClose']").click() # close ads
    time.sleep(1)
else:
    time.sleep(1)

driver.find_element(By.XPATH,"/html/body/header/div/nav/ul/li[10]/a").click() 
time.sleep(6)

if len(driver.find_elements(By.ID, "advertClose"))>0:
    driver.find_element(By.XPATH,"//*[@id='advertClose']").click() # close ads
    time.sleep(1)
else:
    time.sleep(1)

season_no = []
player_name = []
player_position = []
player_nationality = []
player_club = []
player_season = []
player = []

soup = BeautifulSoup(driver.page_source, 'html.parser') #to rerun after change page

seasonSeq = 2
seasoncounter = 1

while seasonSeq <= 4: # previous 3 seasons
    
    driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div/section/div[1]/div[2]").click() # open Season
    time.sleep(1)
    driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div/section/div[1]/ul/li[" + str(seasonSeq) + "]").click() # click from 2020/21
    time.sleep(3)
    
    clubSeq = 2 # reset counter
    
    while clubSeq <= 21:
        
        driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div/section/div[2]/div[2]").click() #open Club
        time.sleep(1)
        driver.find_element(By.XPATH,"/html/body/main/div[2]/div[1]/div/section/div[2]/ul/li[" + str(clubSeq) + "]").click() # click from first Club, total 20 Club every year
        time.sleep(2)
        
        soup = BeautifulSoup(driver.page_source, 'html.parser') #to rerun after change page

        pName_list = soup.find('tbody', attrs={'class':'dataContainer indexSection'}).find_all('a', attrs={'class':'playerName'})
        for pName_items in pName_list:
            player_name.append(pName_items.text)
            current_club = soup.find('div', attrs={'data-dropdown-current':'clubs'})
            player_club.append(current_club.text)
            current_playerSeason = soup.find('div', attrs={'data-dropdown-current':'compSeasons'})
            player_season.append(current_playerSeason.text)
            season_no.append(seasoncounter)

        pPosition_list = soup.find('tbody', attrs={'class':'dataContainer indexSection'}).find_all('tr')

        for pPosition_items in pPosition_list:
            for pPosition_items2 in pPosition_items.find('td',attrs={'class':'hide-s'}):
                player_position.append(pPosition_items2.text)

        pCountry_list = soup.find('tbody', attrs={'class':'dataContainer indexSection'}).find_all('span', attrs={'class':'playerCountry'})

        for pCountry_items in pCountry_list:
            player_nationality.append(pCountry_items.text)
        
        
        clubSeq = clubSeq + 1
        
    seasoncounter = seasoncounter + 1    
    seasonSeq = seasonSeq + 1


In [8]:
player = {'Season No.': season_no,
          'Season': player_season,
          'Club': player_club,
          'Name':player_name,
          'Position':player_position,
          'Nationatility':player_nationality}


In [9]:
pl_player_df = pd.DataFrame(player)
pl_player_df

,Season No.,Season,Club,Name,Position,Nationatility
0,1,2020/21,Arsenal,Bernd Leno,Goalkeeper,Germany
1,1,2020/21,Arsenal,Emiliano Martínez,Goalkeeper,Argentina
2,1,2020/21,Arsenal,Matt Macey,Goalkeeper,England
3,1,2020/21,Arsenal,Héctor Bellerín,Defender,Spain
4,1,2020/21,Arsenal,Kieran Tierney,Defender,Scotland
...,...,...,...,...,...,...
3163,3,2018/19,Wolverhampton Wanderers,Maximilian Kilman,Defender,England
3164,3,2018/19,Wolverhampton Wanderers,Jonny,Defender,Spain
3165,3,2018/19,Wolverhampton Wanderers,Niall Ennis,Forward,England
3166,3,2018/19,Wolverhampton Wanderers,Ethan Ebanks-Landell,Defender,England


In [10]:
pl_player_df.sort_values(by=['Season','Club','Position'])
pl_player_df.to_csv('core/media/data/premierleague_player.csv', index=False)

In [ ]:
##driver.close()